In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

In [2]:
no_of_days_in_advance = 3
no_of_days_rolled = 3

In [3]:
slb_df_train = yf.download('SLB', start='2020-01-01',end='2023-10-10')
slb_df_test = yf.download('SLB', start='2020-10-10',end='2023-10-11')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
slb_df_train.reset_index(inplace=True)
slb_df_test.reset_index(inplace=True)

In [5]:
df_train = slb_df_train[['Date','Open']]
df_train['id'] = [1]*len(df_train)
df_test = slb_df_train[['Date','Open']]
df_test['id'] = [1]*len(df_test)

/var/folders/j7/w6xk0jnj6xx3fy4_9jtdj3x00000gn/T/ipykernel_104/4130263739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['id'] = [1]*len(df_train)
/var/folders/j7/w6xk0jnj6xx3fy4_9jtdj3x00000gn/T/ipykernel_104/4130263739.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['id'] = [1]*len(df_test)


In [6]:
target_vector_train = df_train[no_of_days_rolled+ no_of_days_in_advance:]['Open']
target_vector_test = df_test[no_of_days_rolled + no_of_days_in_advance:]['Open']

In [7]:
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled_train = roll_time_series(df_train, column_id="id", column_sort="Date",min_timeshift=no_of_days_rolled,max_timeshift=no_of_days_rolled)
df_rolled_test = roll_time_series(df_test, column_id="id", column_sort="Date",min_timeshift=no_of_days_rolled,max_timeshift=no_of_days_rolled)

/Users/kaustubh/Workspace/PYTHON_PROJECTS/Projects_2023/PredictingStockPrice/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 20/20 [00:02<00:00,  9.82it/s]
/Users/kaustubh/Workspace/PYTHON_PROJECTS/Projects_2023/PredictingStockPrice/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 20/20 [00:01<00:00, 11.36it/s]


In [8]:
from tsfresh import extract_features
df_features_train = extract_features(df_rolled_train, column_id="id", column_sort="Date")
df_features_test = extract_features(df_rolled_test, column_id="id", column_sort="Date")

Feature Extraction: 100%|██████████| 20/20 [00:04<00:00,  4.03it/s]


In [9]:
target_vector_train_list = list(target_vector_train) + [-1]*no_of_days_in_advance
target_vector_test_list = list(target_vector_test) + [-1]*no_of_days_in_advance


In [10]:
df_features_train['target'] = target_vector_train_list
df_features_test['target'] = target_vector_test_list

In [11]:
df_features_train = df_features_train[df_features_train['target'] > 0]
df_features_test = df_features_test[df_features_test['target'] > 0]

In [12]:
df_features_train.dropna(axis = 1,inplace=True)
df_features_test.dropna(axis = 1,inplace=True)

In [13]:
important_features_based_on_train = df_features_train.corr()['target'].abs().sort_values(ascending=False)[0:30]
# important_features_based_on_train = important_features_based_on_train.drop(important_features_based_on_train.index[0])

In [14]:
df_for_ml_model_train = df_features_train[list(important_features_based_on_train.reset_index()['index'])].reset_index()
del df_for_ml_model_train['level_0']
del df_for_ml_model_train['level_1']


In [17]:
df_for_ml_model_test = df_features_test[list(important_features_based_on_train.reset_index()['index'])].reset_index()
del df_for_ml_model_test['level_0']
del df_for_ml_model_test['level_1']

In [18]:
import numpy as np
y = df_for_ml_model_train.iloc[:,0]
x = df_for_ml_model_train.iloc[:,1: ]
y_test = df_for_ml_model_test.iloc[:,0]
x_test = df_for_ml_model_test.iloc[:,1: ]


In [19]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=3,random_state=0)
regressor.fit(x,y)

RandomForestRegressor(n_estimators=3, random_state=0)

In [20]:
y_test_pred = regressor.predict(x_test)

In [21]:
df_prediction = pd.DataFrame(zip(list(y_test_pred),list(y_test)),columns=['Predicted','Real Data'])
df_prediction['Date'] = slb_df_test['Date']

In [22]:
import plotly.express as px
fig = px.scatter(df_prediction,x='Date',y=['Predicted','Real Data'],title='SLB STOCK')
# fig = fig.add_scatter(df_prediction,x='Date',y='Real Data',title='Real')
fig.show()

In [24]:
df_prediction['Error'] = df_prediction['Predicted'] - df_prediction['Real Data']

In [25]:
import plotly.express as px
fig = px.scatter(df_prediction,x='Date',y=['Error'],title='SLB STOCK')
# fig = fig.add_scatter(df_prediction,x='Date',y='Real Data',title='Real')
fig.show()

In [26]:
list_change_similarity =[]
for i in range(0,len(df_prediction)-1):
    change_in_prediction=(df_prediction['Predicted'].iloc[i+1]- df_prediction['Predicted'].iloc[i])
    change_in_real=(df_prediction['Real Data'].iloc[i+1]- df_prediction['Real Data'].iloc[i])

    if(change_in_prediction > 0 and change_in_real > 0):
        list_change_similarity.append(1)
    elif(change_in_prediction < 0 and change_in_real < 0):
        list_change_similarity.append(1)
    else:
        list_change_similarity.append(-1)



In [32]:
print(sum(list_change_similarity))
print(len(list_change_similarity))

448
942


In [33]:
745/942

0.7908704883227177